# 0.- Chilton f0F2 data - initial cleansing
## June 2022
### Jose M Albornoz

In this notebook we perform initial cleansing and formatting of a dataset containing hourly values of the F2 critical frequency (f0f2) measured at Chilton, UK.

# Import necessary modules

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import cm
style.use('fivethirtyeight')
%matplotlib inline

import seaborn as sns

import random
import time

RANDOM_STATE = 801
pd.options.mode.chained_assignment = None

# maximum number of rdataframe ows and columns displayed
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)

# 1.- Data Provenance

The dataset employed in this notebook was obtained from the UK Solar System Data Centre portal (https://www.ukssdc.ac.uk/wdcc1/ionosondes/secure/iono_data.shtml). f0F2 data from the Chilton ionosonde was retrieved, covering from 20/03/1995 to 1995 up to 31/03/2009. Times are expressed in UT+00

# 2.- Data Ingestion

In [2]:
df_0 = pd.read_csv('Data/f0f2_rawData_v1.txt', delimiter=r"\s+", usecols=['YYYYMMDD', 'HHMMSS', '00', 'SLASH'])

In [3]:
df_0.shape

(102441, 4)

In [4]:
df_0.head()

,YYYYMMDD,HHMMSS,00,SLASH
0,19950320,150000,/,NaN
1,19950320,160000,57,/
2,19950320,170000,57,/
3,19950320,180000,53,/
4,19950320,190000,48,/


In [5]:
df_0.drop('SLASH', axis=1, inplace=True)

In [6]:
df_0.dtypes

YYYYMMDD    object
HHMMSS      object
00          object
dtype: object

# 3.- Data Cleansing

## 3.1.-  Remove rows containing text (headers) 

In [7]:
indexNames1 = df_0[df_0['YYYYMMDD'] == "Time"].index

# Delete these row indexes from dataFrame
df_0.drop(indexNames1 , inplace=True)

In [8]:
indexNames2 = df_0[df_0['YYYYMMDD'] == "YYYYMMDD"].index

# Delete these row indexes from dataFrame
df_0.drop(indexNames2 , inplace=True)

## 3.2.-  Join date and time columns to create timestamp column

In [9]:
df_0['Timestamp0'] = df_0[['YYYYMMDD', 'HHMMSS']].apply(lambda x: ' '.join(x), axis=1)

In [10]:
df_0.head()

,YYYYMMDD,HHMMSS,00,Timestamp0
0,19950320,150000,/,19950320 150000
1,19950320,160000,57,19950320 160000
2,19950320,170000,57,19950320 170000
3,19950320,180000,53,19950320 180000
4,19950320,190000,48,19950320 190000


## 3.3.-  Convert timestamp column to datetime, drop unnecessary columns

In [11]:
df_0['Timestamp'] = pd.to_datetime(df_0['Timestamp0'], format='%Y%m%d %H%M%S')

In [12]:
df_0.columns

Index(['YYYYMMDD', 'HHMMSS', '00', 'Timestamp0', 'Timestamp'], dtype='object')

In [13]:
df_1 = df_0.drop(['YYYYMMDD', 'HHMMSS', 'Timestamp0'], axis=1)

In [14]:
df_1.head()

,00,Timestamp
0,/,1995-03-20 15:00:00
1,57,1995-03-20 16:00:00
2,57,1995-03-20 17:00:00
3,53,1995-03-20 18:00:00
4,48,1995-03-20 19:00:00


In [15]:
df_1.columns = ['f0f2x10', 'Timestamp']

## 3.4.-  Eliminate rows with duplicated timestamps

In [16]:
df_1.shape

(102105, 2)

In [17]:
duplicated_rows = df_1['Timestamp'].duplicated()

In [18]:
duplicated_rows.sum()

4

In [19]:
df_2 = df_1[~duplicated_rows]

In [20]:
df_2.shape

(102101, 2)

## 3.5.- Replace non-numerical values in f0f2x10

### 3.5.1.- Replace values such as ''38/" in f0f2x10

In [21]:
df_2[df_2['f0f2x10'].str.match("^\d+\D")] 

,f0f2x10,Timestamp
3814,38/,1995-09-01 00:00:00
3815,35/,1995-09-01 01:00:00
3816,35/,1995-09-01 02:00:00
3817,33/,1995-09-01 03:00:00
3818,31/,1995-09-01 04:00:00
...,...,...
102436,49/,2009-03-30 18:00:00
102437,24/,2009-03-31 00:00:00
102438,36/,2009-03-31 06:00:00
102439,49/,2009-03-31 12:00:00


In [22]:
indexes = df_2[df_2['f0f2x10'].str.match("^\d+\D")].index

In [23]:
df_2.loc[indexes,'f0f2x10'] = df_2.loc[indexes].f0f2x10.str.extract('(^\d+)').values

In [24]:
df_2.loc[indexes]

,f0f2x10,Timestamp
3814,38,1995-09-01 00:00:00
3815,35,1995-09-01 01:00:00
3816,35,1995-09-01 02:00:00
3817,33,1995-09-01 03:00:00
3818,31,1995-09-01 04:00:00
...,...,...
102436,49,2009-03-30 18:00:00
102437,24,2009-03-31 00:00:00
102438,36,2009-03-31 06:00:00
102439,49,2009-03-31 12:00:00


### 3.5.2.- Replace remaining non-numerical values in f0f2x10

In [25]:
df_2[df_2['f0f2x10'].str.match("\D+")] 

,f0f2x10,Timestamp
0,/,1995-03-20 15:00:00
5,/,1995-03-20 20:00:00
14,/,1995-03-21 05:00:00
15,/,1995-03-21 06:00:00
16,/,1995-03-21 07:00:00
17,/,1995-03-21 08:00:00
18,/,1995-03-21 09:00:00
19,/,1995-03-21 10:00:00
60,/,1995-03-23 03:00:00
61,/,1995-03-23 04:00:00


In [26]:
indexes = df_2[df_2['f0f2x10'].str.match("\D+")].index

In [27]:
df_2.loc[indexes,'f0f2x10'] = np.nan

## 3.6.-  Proportion of missing values

In [28]:
df_2.isna().sum()

f0f2x10      3318
Timestamp       0
dtype: int64

In [29]:
df_2.isna().sum()*100/len(df_2)

f0f2x10      3.249723
Timestamp    0.000000
dtype: float64

## 3.7.-  Express f0F2 in MHz

In [30]:
df_2['f0f2'] = df_2["f0f2x10"].astype(float)/10.

In [31]:
df_2.drop('f0f2x10', axis=1, inplace=True)

In [32]:
df_2.head()

,Timestamp,f0f2
0,1995-03-20 15:00:00,NaN
1,1995-03-20 16:00:00,5.7
2,1995-03-20 17:00:00,5.7
3,1995-03-20 18:00:00,5.3
4,1995-03-20 19:00:00,4.8


# 4.-  Write formatted dataframe to disk

In [33]:
df_2.to_csv("Data/Chiltonf0f2_Clean_v1.csv", index=False)